# Bucur Robert - Adrian
## Grupa 10LF381
## Genetic Algorithm - BucurRobert-381-ga.ipynb

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, List
import bitstring
import random

## Definirea functiei pt calculul maximului

In [2]:
def f(x, y):
    return np.abs((10 * x ** 2 * y - 5 * x ** 2 - 4 * y ** 2 - x ** 4 - 2 * y ** 4) / 2) + 1

## Definirea hiperparametrilor

In [3]:
x_inf_domain = -20      # marginea inferioara a domeniului functiei
x_sup_domain = 20       # marginea superioara a domeniului functiei
y_inf_domain = -5       # marginea inferioara a codomeniului functiei
y_sup_domain = 5        # marginea superioara a codomeniului functiei
n = 500                 # indivizi per generatie
m = 60                  # numar maxim de iteratii
k_x = 8                 # gene x per cromozom
k_y = 16                # gene y per cromozom
prob_imperechere = 0.1  # probabilitate imperechere
prob_mutatie = 0.01     # probabilitate mutatie

## Definire de functii utile

In [4]:
def cromozom(x_string, y_string):
    bitstring_x = bitstring.BitArray(bin=x_string)
    bitstring_y = bitstring.BitArray(bin=y_string)
    v_c_x = bitstring_x.uint
    v_c_y = bitstring_y.uint
    x = x_inf_domain + v_c_x * ((x_sup_domain - x_inf_domain) / (2 ** k_x - 1))
    y = y_inf_domain + v_c_y * ((y_sup_domain - y_inf_domain) / (2 ** k_y - 1))
    return x, y, f(x, y)

In [5]:
def generatie(x_generatie, y_generatie):
    x = np.zeros((n))
    y = np.zeros((n))
    value = np.zeros((n))
    dimensiune_generatie = x_generatie.shape[0]
    for i in range(0, dimensiune_generatie):
        x[i], y[i], value[i] = cromozom(x_generatie[i], y_generatie[i])
    return x, y, value

## Crearea unei populatii initiale de cromozomi

In [6]:
def random_bs(k):
    return "".join(str(random.randint(0, 1)) for x in range(0,k))
populatie_x = np.empty([m, n], dtype='object')
populatie_y = np.empty([m, n], dtype='object')
for i in range(0, n):
    populatie_x[0, i] = random_bs(k_x)
    populatie_y[0, i] = random_bs(k_y)

## Incrucisarea

In [7]:
def incrucisare(generatie, k):
    candidati = np.empty(0)
    nr = np.random.uniform(0, 1, n)

    for i in range(0, n):
        if nr[i] < prob_imperechere:
            candidati = np.append(candidati, i)

    if candidati.shape[0] >= 2:
        if candidati.shape[0] % 2 != 0:
            candidati = candidati[0:-1]

        for i in range(0, candidati.shape[0], 2):
            t = int(np.random.uniform(1, k-1, 1))
            c = int(candidati[i])
            generatie[c] = generatie[c][:t] + generatie[c+1][t:]
            generatie[c+1] = generatie[c+1][:t] + generatie[c][t:]

    return generatie

## Mutatia

In [8]:
def mutatie(generatie, k):
    nr = np.random.uniform(0, 1, n*k)
    i = 0

    for cromozom_i, cromozom in enumerate(generatie):
        for gena_i, gena in enumerate(cromozom):
            if nr[i] < prob_mutatie:
                bs = generatie[cromozom_i]
                generatie[cromozom_i] = bs[:gena_i] + ('1' if bs[gena_i] == '0' else '1') + bs[gena_i+1:]
            i += 1

    return generatie

# Implementarea algoritmului

In [9]:
g_index = 1
while g_index < m:
    g_xs, g_ys, g_values = generatie(populatie_x[g_index - 1], populatie_y[g_index -1])
    g_sum = np.sum(g_values)
    g_p = g_values / g_sum
    g_q = np.cumsum(g_p)
    nr = np.random.uniform(0, 1, n)
    for index in range(0, n):
        for q_index, q_value in enumerate(g_q):
            if nr[index] <= q_value:
                populatie_x[g_index, index] = populatie_x[g_index - 1, q_index]
                populatie_y[g_index, index] = populatie_y[g_index - 1, q_index]
                break
    populatie_x[g_index] = incrucisare(populatie_x[g_index], k_x)
    populatie_y[g_index] = incrucisare(populatie_y[g_index], k_y)
    populatie_x[g_index] = mutatie(populatie_x[g_index], k_x)
    populatie_y[g_index] = mutatie(populatie_y[g_index], k_y)
    g_index += 1

## Crearea fisierului rezultat

In [10]:
file = open("ga-result.txt", 'w')

for g_index in range(0,m):
    file.write(f"#inceput date generatie {g_index}\n")
    
    g_xs,g_ys,g_values = generatie(populatie_x[g_index], populatie_y[g_index])
    for index in range(0,n):
        file.write(f"cromozom: x={g_xs[index]}, y={g_ys[index]}\nvaloare: {g_values[index]}\n")

    file.write(f"#sfarsit date generatie {g_index}\n")

file.close()